In [4]:
!pip install pandas

    100% |████████████████████████████████| 14.7MB 553kB/s ta 0:00:01
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np

In [2]:
koubei_train = pd.read_csv("data/ijcai2016_koubei_train.csv")
koubei_train.head(5)

,use_ID,mer_ID,loc_ID,time
0,1027765,4822,172,20151028
1,141398,4822,172,20151115
2,141398,4822,172,20151005
3,1027765,4822,172,20151026
4,141398,4822,172,20151011


In [3]:
koubei_train.shape

(1081724, 4)

In [30]:
dict = {}
exist_dict = {}
mer_loc_count = {}
merchant_count = 0
merchant_dict = {}
open_interval_dict = {}
time_interval_dict = {}
openning_time_dict = {}
last_time_dict = {}
location_count = {}
count = 2000
# sort by time
koubei_train.sort_values("time", inplace=True)

In [31]:
koubei_train.head(20)

,use_ID,mer_ID,loc_ID,time
400965,727382,210,410,20150701
271666,958880,5231,337,20150701
275360,1976144,5231,337,20150701
275364,1976144,5231,337,20150701
401005,727382,210,410,20150701
401015,727382,210,410,20150701
400985,674494,210,410,20150701
400946,727382,210,410,20150701
271665,958880,5231,337,20150701
400979,674494,210,410,20150701


In [129]:
for index, row in koubei_train.iterrows():
    # get location count
    if row["loc_ID"] not in location_count:
        location_count[row["loc_ID"]] = 1
    else:
        location_count[row["loc_ID"]] += 1
    
    # get merchant location dict & open interval & time interval
    if (row["mer_ID"], row["loc_ID"]) not in mer_loc_count:
        mer_loc_count[(row["mer_ID"], row["loc_ID"])] = 1
        open_interval_dict[(row["mer_ID"], row["loc_ID"])] = row["time"]
        time_interval_dict[(row["mer_ID"], row["loc_ID"], row["time"])] = 0
    else:
        mer_loc_count[(row["mer_ID"], row["loc_ID"])] += 1
        # get time interval between this (m,l) and open_interval
        time_interval_dict[(row["mer_ID"], row["loc_ID"], row["time"])] = row["time"] - open_interval_dict[(row["mer_ID"], row["loc_ID"])]
    
    # get openning time
    if (row["mer_ID"], row["loc_ID"]) not in openning_time_dict:
        openning_time_dict[(row["mer_ID"], row["loc_ID"])] = row["time"]
    else:
        pass
    # get last purchase time (increased by time, just update time to get last purchase time)
    last_time_dict[(row["mer_ID"], row["loc_ID"])] = row["time"]

#     count -= 1
#     if count <= 0:
#         break


In [130]:
koubei_train.head(5)

,use_ID,mer_ID,loc_ID,time
400965,727382,210,410,20150701
271666,958880,5231,337,20150701
275360,1976144,5231,337,20150701
275364,1976144,5231,337,20150701
401005,727382,210,410,20150701


In [131]:
# mer_loc_count
count = 5
for (k, v) in mer_loc_count.items():
    print(k, v)
    count -= 1
    if count < 0:
        break

(210, 410) 189
(5231, 337) 6338
(625, 130) 158
(216, 407) 182
(5231, 197) 4669
(5231, 254) 2503


In [132]:
# location_count
count = 5
for (k, v) in location_count.items():
    print(k, v)
    count -= 1
    if count < 0:
        break

410 5357
337 24988
130 3873
407 527
197 19205
254 14082


In [146]:
# mer_loc_popularity dict
mer_loc_popularity = {}
for (k, v) in mer_loc_count.items():
    ratio = v / location_count[(k[1])]
    mer_loc_popularity[k] = ratio

In [147]:
# date interval 
from datetime import date
def date_interval(date1, date2):

    date1 = date(int(str(date1)[:4]), int(str(date1)[4:6]), int(str(date1)[6:]))
    date2 = date(int(str(date2)[:4]), int(str(date2)[4:6]), int(str(date2)[6:]))
    return abs(date2 - date1).days
print(date_interval(20180910, 20170819))

387


In [148]:
# average purchase interval
avg_pur_interval = {}
for (k, v) in mer_loc_count.items():
    openning_date = int(openning_time_dict[k])
    last_date = int(last_time_dict[k])
    average_interval = (date_interval(last_date, openning_date) / v)
    avg_pur_interval[k] = average_interval
#     print("openning_date: {}, last_date: {}, mer_loc_count: {}, average_interval: {}".format(openning_date, last_date, v, average_interval))


In [142]:
import pandas as pd
df = pd.DataFrame(columns=['mer_id', 'loc_id', 'mer_loc_records', 'popularity', 'openning_date', 'last_purchase_date', 'average_purchase_interval'])

In [143]:
df.head()

,mer_id,loc_id,mer_loc_records,popularity,openning_date,last_purchase_date,average_purchase_interval


In [149]:
mer_loc_id = 1
for (k, v) in mer_loc_count.items():
    mer_id = k[0]
    loc_id = k[1]
    mer_loc_records = v
    popularity = mer_loc_popularity[k]
    openning_date = openning_time_dict[k]
    last_purchase_date = last_time_dict[k]
    average_purchase_interval = avg_pur_interval[k]
    df = df.append(pd.DataFrame(
        {
            'mer_id': mer_id,
            'loc_id': loc_id,
            'mer_loc_records': mer_loc_records,
            'popularity': popularity,
            'openning_date': openning_date,
            'last_purchase_date': last_purchase_date,
            'average_purchase_interval': average_purchase_interval
        }, index = [mer_loc_id]))
    mer_loc_id += 1


(210, 410)
(5231, 337)
(625, 130)
(216, 407)
(5231, 197)
(5231, 254)
(5231, 461)
(5231, 371)
(2641, 441)
(5231, 17)
(5231, 69)
(2156, 348)
(5231, 348)
(5231, 308)
(5231, 265)
(5231, 63)
(5231, 60)
(5231, 246)
(5231, 91)
(5231, 112)
(5231, 19)
(2641, 52)
(3090, 84)
(5231, 183)
(5231, 163)
(4565, 461)
(5231, 172)
(4565, 116)
(2142, 114)
(4565, 188)
(7269, 429)
(8960, 461)
(5231, 490)
(4565, 451)
(4174, 360)
(5231, 42)
(5609, 106)
(4565, 132)
(5231, 27)
(625, 337)
(30, 78)
(5231, 326)
(4269, 499)
(5231, 57)
(4565, 182)
(8248, 346)
(7661, 355)
(5223, 162)
(2641, 402)
(8758, 69)
(4565, 19)
(2455, 125)
(7184, 355)
(4565, 437)
(5231, 77)
(7786, 410)
(8758, 192)
(9921, 461)
(4269, 349)
(5231, 146)
(8374, 132)
(5074, 178)
(3090, 129)
(8758, 88)
(3869, 178)
(6045, 157)
(4565, 249)
(8758, 308)
(7926, 132)
(5231, 199)
(7830, 172)
(6900, 249)
(4074, 76)
(4565, 378)
(4565, 410)
(2641, 95)
(8758, 407)
(8758, 163)
(4565, 114)
(2641, 40)
(8628, 239)
(508, 254)
(7383, 410)
(2156, 146)
(7184, 341)
(484, 

(376, 88)
(9397, 355)
(1758, 455)
(186, 360)
(3121, 480)
(7755, 470)
(5307, 155)
(2095, 294)
(9029, 72)
(4961, 348)
(4960, 230)
(4697, 52)
(1087, 262)
(5300, 438)
(4674, 441)
(3440, 272)
(131, 461)
(49, 312)
(3893, 102)
(150, 76)
(3304, 378)
(5755, 26)
(9191, 358)
(4061, 360)
(3230, 272)
(8558, 78)
(8671, 172)
(1073, 360)
(5530, 33)
(5530, 76)
(8640, 111)
(5530, 262)
(1391, 88)
(3498, 84)
(5530, 158)
(3066, 190)
(3971, 364)
(5530, 216)
(625, 254)
(2699, 6)
(7212, 89)
(1492, 442)
(7682, 126)
(5530, 268)
(3498, 188)
(5530, 239)
(9127, 490)
(477, 76)
(6222, 91)
(5530, 114)
(2150, 17)
(5530, 141)
(1450, 88)
(3066, 346)
(468, 188)
(8028, 185)
(2901, 235)
(8580, 155)
(5530, 441)
(5530, 277)
(278, 88)
(5351, 82)
(2822, 106)
(622, 253)
(5530, 89)
(278, 182)
(3538, 84)
(9178, 88)
(7398, 17)
(278, 441)
(8487, 340)
(7252, 17)
(5232, 6)
(5139, 172)
(278, 17)
(6956, 76)
(270, 355)
(8084, 195)
(8254, 138)
(5530, 176)
(728, 60)
(2060, 132)
(3066, 78)
(8799, 190)
(1973, 378)
(1970, 178)
(6352, 63)
(55

(2447, 355)
(5016, 206)
(2381, 360)
(2732, 364)
(831, 234)
(1270, 52)
(778, 249)
(5495, 88)
(2945, 339)
(5651, 216)
(5206, 378)
(1510, 378)
(9702, 190)
(1887, 322)
(6618, 498)
(5773, 23)
(3100, 23)
(473, 17)
(2254, 245)
(5210, 82)
(9883, 423)
(8739, 63)
(4564, 52)
(8397, 17)
(3255, 84)
(8460, 216)
(354, 216)
(4089, 239)
(7066, 162)
(2311, 17)
(7157, 360)
(201, 72)
(3818, 308)
(5727, 6)
(7840, 325)
(6081, 133)
(7279, 474)
(8427, 396)
(2929, 360)
(334, 188)
(6187, 52)
(5081, 372)
(2715, 254)
(4712, 490)
(1921, 159)
(6697, 254)
(7574, 91)
(7068, 76)
(630, 441)
(498, 133)
(4838, 44)
(3932, 94)
(5257, 91)
(5335, 378)
(2328, 265)
(1217, 468)
(486, 239)
(2749, 378)
(4280, 335)
(8497, 348)
(628, 341)
(1226, 337)
(7261, 254)
(2156, 254)
(2014, 253)
(110, 178)
(5688, 355)
(4894, 378)
(8484, 178)
(5243, 17)
(8427, 174)
(3618, 190)
(8619, 461)
(412, 235)
(6908, 6)
(633, 364)
(2831, 119)
(3808, 52)
(8539, 213)
(153, 190)
(2831, 123)
(7155, 102)
(955, 211)
(473, 308)
(8701, 172)
(4852, 26)
(8961, 16

(8331, 337)
(2761, 360)
(8489, 45)
(83, 56)
(95, 366)
(5262, 375)
(3214, 39)
(2595, 360)
(6160, 402)
(2369, 441)
(8966, 402)
(7764, 40)
(8106, 294)
(4164, 294)
(6534, 82)
(6297, 88)
(7478, 88)
(2622, 146)
(6841, 477)
(6552, 275)
(9997, 327)
(2622, 49)
(4788, 163)
(4303, 333)
(9901, 333)
(6400, 6)
(549, 83)
(8533, 83)
(6803, 47)
(6465, 157)
(7932, 378)
(1300, 355)
(4932, 104)
(3329, 327)
(5295, 327)
(1397, 91)
(4338, 272)
(9043, 268)
(7671, 352)
(8257, 254)
(5133, 416)
(9373, 34)
(5303, 146)
(892, 6)
(2248, 50)
(282, 155)
(1514, 188)
(2695, 73)
(2831, 62)
(2703, 75)
(541, 349)
(4817, 40)
(4648, 378)
(2285, 360)
(6752, 135)
(6555, 34)
(1437, 135)
(5670, 360)
(8021, 159)
(381, 75)
(6034, 378)
(4822, 172)
(9899, 216)
(503, 268)
(5370, 32)
(138, 175)
(6101, 445)
(676, 11)
(5789, 303)
(5583, 355)
(4139, 17)
(552, 168)
(9590, 172)
(48, 272)
(7880, 73)
(9243, 303)
(8590, 249)
(7545, 216)
(5049, 11)
(4059, 404)
(4360, 87)
(7263, 106)
(913, 88)
(8308, 416)
(4243, 17)
(5894, 162)
(8822, 327)
(348

(1794, 480)
(3848, 142)
(7976, 309)
(7798, 85)
(8274, 368)
(1417, 152)
(9723, 137)
(1297, 68)
(1835, 378)
(3153, 268)
(1123, 145)
(4856, 362)
(268, 92)
(7046, 94)
(9809, 104)
(6583, 75)
(9068, 192)
(6743, 360)
(7691, 272)
(9506, 228)
(2159, 445)
(3830, 9)
(7948, 388)
(824, 75)
(8222, 430)
(8646, 30)
(2315, 9)
(4116, 349)
(9851, 30)
(546, 372)
(7790, 378)
(4245, 245)
(4536, 322)
(9242, 6)
(421, 6)
(280, 114)
(6960, 372)
(2110, 60)
(3198, 349)
(4745, 349)
(9486, 345)
(311, 189)
(6834, 349)
(2217, 340)
(743, 393)
(8246, 99)
(370, 158)
(302, 322)
(6052, 49)
(4056, 346)
(1729, 23)
(9646, 211)
(212, 339)
(1618, 340)
(5616, 6)
(9974, 11)
(3069, 193)
(996, 463)
(4273, 318)
(6224, 6)
(664, 20)
(7423, 262)
(3200, 243)
(2787, 462)
(5430, 102)
(5123, 172)
(4225, 272)
(1668, 326)
(1679, 303)
(1416, 13)
(448, 132)
(4762, 107)
(8218, 294)
(4457, 349)
(2870, 284)
(3285, 461)
(5083, 162)
(352, 75)
(3715, 80)
(206, 372)
(9489, 98)
(2586, 324)
(3462, 115)
(9061, 214)
(5548, 216)
(3987, 288)
(213, 11)
(77

(5555, 33)
(7597, 33)
(312, 272)
(3993, 172)
(7229, 133)
(1581, 152)
(9640, 146)
(4456, 462)
(3263, 133)
(4456, 92)
(1099, 340)
(6634, 58)
(1260, 245)
(44, 348)
(1229, 245)
(5475, 441)
(4456, 315)
(2619, 166)
(2162, 348)
(3379, 137)
(9936, 220)
(7663, 178)
(423, 132)
(493, 125)
(7132, 360)
(6660, 157)
(3824, 287)
(4456, 270)
(420, 95)
(4456, 406)
(7175, 333)
(7370, 223)
(7129, 488)
(2937, 341)
(4364, 104)
(2631, 33)
(176, 216)
(5359, 104)
(9144, 272)
(8432, 164)
(2100, 481)
(3063, 457)
(9592, 138)
(7779, 33)
(9543, 88)
(2651, 88)
(6863, 305)
(4526, 390)
(846, 91)
(5655, 84)
(6844, 146)
(4212, 358)
(8997, 21)
(1521, 146)
(2758, 474)
(6808, 138)
(9077, 352)
(163, 296)
(3241, 445)
(1436, 341)
(4445, 52)
(1029, 474)
(4318, 445)
(6541, 80)
(129, 102)
(4248, 349)
(5634, 349)
(493, 190)
(3394, 323)
(2647, 315)
(8885, 319)
(912, 306)
(2957, 214)
(2484, 63)
(8268, 6)
(94, 162)
(360, 422)
(4951, 56)
(524, 25)
(4737, 296)
(8847, 162)
(4543, 278)
(8128, 88)
(4386, 138)
(100, 43)
(5803, 138)
(3528,

(1906, 324)
(7568, 360)
(9457, 32)
(9894, 318)
(284, 155)
(7640, 379)
(5692, 322)
(4167, 220)
(64, 172)
(635, 20)
(7352, 17)
(9824, 399)
(7343, 214)
(5809, 124)
(7, 446)
(253, 322)
(5697, 362)
(2568, 355)
(1353, 490)
(9267, 182)
(9722, 98)
(3912, 408)
(8928, 337)
(3367, 24)
(5151, 217)
(7040, 490)
(1143, 93)
(3124, 40)
(7823, 355)
(208, 167)
(8435, 214)
(3269, 34)
(1247, 325)
(6635, 448)
(2223, 172)
(6709, 172)
(8956, 32)
(8748, 33)
(1, 30)
(6183, 262)
(6290, 216)
(3863, 88)
(5989, 245)
(6714, 214)
(2673, 254)
(9918, 243)
(929, 178)
(3637, 324)
(8255, 238)
(1284, 125)
(4256, 172)
(1791, 105)
(7181, 26)
(6252, 178)
(2964, 34)
(9559, 358)
(4791, 213)
(2275, 105)
(1293, 197)
(263, 172)
(1371, 8)
(7935, 93)
(3500, 9)
(3249, 217)
(1827, 84)
(1061, 495)
(9743, 88)
(4897, 161)
(1667, 11)
(5650, 20)
(1423, 360)
(7945, 327)
(534, 249)
(1065, 154)
(8962, 352)
(6870, 333)
(1661, 102)
(8341, 461)
(1134, 121)
(4741, 262)
(4080, 80)
(908, 404)
(8808, 490)
(4736, 88)
(3595, 88)
(1396, 213)
(5719, 300

(1111, 287)
(6321, 115)
(8876, 207)
(4625, 189)
(4713, 480)
(578, 91)
(2532, 301)
(1574, 345)
(572, 257)
(9509, 159)
(5294, 183)
(8455, 461)
(4156, 83)
(7555, 377)
(4090, 130)
(1390, 40)
(4492, 34)
(7218, 15)
(2673, 322)
(2816, 162)
(3963, 32)
(8656, 56)
(8231, 6)
(4397, 61)
(961, 83)
(8161, 343)
(7376, 132)
(9859, 132)
(1554, 6)
(9653, 38)
(113, 84)
(8875, 345)
(5294, 88)
(7246, 410)
(4793, 467)
(629, 481)
(6664, 57)
(7069, 33)
(6737, 315)
(3325, 30)
(5902, 178)
(3495, 213)
(2507, 178)
(8336, 81)
(593, 81)
(2911, 448)
(2652, 165)
(6178, 188)
(677, 482)
(4108, 358)
(6223, 146)
(5862, 21)
(1169, 228)
(6157, 110)
(2606, 78)
(2382, 327)
(9933, 66)
(3408, 474)
(4051, 91)
(4180, 358)
(144, 114)
(2849, 362)
(8447, 60)
(7183, 62)
(7814, 355)
(18, 448)
(3251, 203)
(1714, 165)
(2271, 162)
(1911, 32)
(4185, 214)
(8795, 250)
(8987, 8)
(478, 480)
(5294, 163)
(362, 448)
(6332, 202)
(5596, 35)
(4503, 445)
(3129, 434)
(2606, 60)
(2549, 162)
(2298, 287)
(173, 162)
(39, 406)
(6733, 459)
(6768, 50)
(570

(3896, 343)
(7071, 177)
(4048, 239)
(794, 344)
(9073, 69)
(7092, 114)
(9529, 58)
(9167, 77)
(2278, 207)
(1941, 263)
(2655, 227)
(594, 352)
(594, 484)
(8423, 355)
(9167, 248)
(9069, 404)
(4957, 438)
(4977, 32)
(6375, 283)
(9027, 132)
(8421, 308)
(118, 337)
(7436, 378)
(25, 346)
(2659, 277)
(4924, 277)
(3469, 141)
(8091, 348)
(4188, 330)
(9167, 19)
(9475, 108)
(6348, 88)
(7347, 59)
(2137, 360)
(2606, 142)
(8931, 441)
(8232, 410)
(9864, 101)
(3687, 461)
(2312, 349)
(6591, 327)
(2234, 56)
(9407, 109)
(348, 366)
(6799, 68)
(9421, 453)
(5866, 68)
(7533, 346)
(9407, 482)
(609, 490)
(8784, 381)
(4975, 461)
(3838, 457)
(8602, 382)
(6012, 61)
(7602, 360)
(4906, 263)
(9143, 382)
(435, 6)
(4516, 382)
(4266, 263)
(9322, 263)
(7591, 457)
(2172, 360)
(1285, 82)
(4689, 61)
(7768, 50)
(2883, 60)
(294, 445)
(615, 40)
(5395, 40)
(653, 445)
(615, 404)
(7878, 421)
(6379, 421)
(4667, 170)
(4299, 13)
(2704, 161)
(8548, 86)
(2606, 202)
(4920, 178)
(4292, 243)
(615, 115)
(7200, 132)
(402, 52)
(406, 476)
(1059,

(4945, 435)
(4491, 429)
(6243, 6)
(3741, 6)
(6928, 228)
(3074, 197)
(6087, 114)
(9571, 6)
(1834, 410)
(3757, 82)
(5059, 204)
(9074, 378)
(5839, 483)
(7431, 436)
(9063, 483)
(8332, 483)
(8266, 483)
(7442, 448)
(2270, 155)
(8043, 245)
(2866, 6)
(897, 261)
(3204, 337)
(9220, 378)
(2915, 378)
(7500, 490)
(9775, 165)
(1520, 414)
(5397, 7)
(4208, 337)
(5890, 256)
(3975, 256)
(1107, 378)
(7572, 390)
(1633, 197)
(1024, 456)
(8335, 49)
(1908, 189)
(2590, 92)
(3785, 345)
(9358, 189)
(1028, 283)
(3778, 190)
(2346, 194)
(810, 194)
(4998, 275)
(3877, 275)
(9435, 25)
(4651, 339)
(876, 275)
(3947, 17)
(338, 91)
(687, 327)
(7886, 358)
(3679, 358)
(1447, 500)
(1772, 85)
(1208, 358)
(5956, 273)
(3994, 330)
(1137, 293)
(4990, 293)
(5639, 428)
(5467, 333)
(2361, 333)
(9649, 216)
(9520, 38)
(9418, 216)
(3756, 497)
(759, 387)
(2229, 407)
(1728, 121)
(227, 177)
(4027, 216)
(9675, 25)
(4651, 446)
(2895, 339)
(5377, 313)
(3473, 313)
(6529, 292)
(7372, 372)
(8335, 125)
(2566, 270)
(2323, 125)
(4534, 124)
(8959,

In [150]:
df.to_csv("mer_loc_data.csv", encoding='utf-8')
df.head(20)

,mer_id,loc_id,mer_loc_records,popularity,openning_date,last_purchase_date,average_purchase_interval
1,210,410,189,0.035281,20150701,20151130,0.804233
2,5231,337,6338,0.253642,20150701,20151130,0.023982
3,625,130,158,0.040795,20150701,20151130,0.962025
4,216,407,182,0.345351,20150701,20151130,0.835165
5,5231,197,4669,0.243114,20150702,20151130,0.032341
6,5231,254,2503,0.177745,20150702,20151130,0.060328
1,210,410,189,0.035281,20150701,20151130,0.804233
2,5231,337,6338,0.253642,20150701,20151130,0.023982
3,625,130,158,0.040795,20150701,20151130,0.962025
4,216,407,182,0.345351,20150701,20151130,0.835165


In [127]:
!pip install sklearn

    100% |████████████████████████████████| 7.8MB 12kB/s ta 0:00:014
  Running setup.py bdist_wheel for sklearn ... done
  Stored in directory: /Users/starwe/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.externals import joblib
import matplotlib.pyplot as plt
%matplotlib inline

file = open('mer_loc_data.csv' , 'r')

data = [line.strip().split(',') for line in file]
feature = np.array([[float(x) for x in row[1:]] for row in data[1:]])

clf = KMeans(n_clusters=30)
s = clf.fit(feature)
print(s)

In [90]:
pd.get_dummies(koubei_train["use_ID"].head(10))

,674494,727382,930608,958880
400965,0,1,0,0
400950,1,0,0,0
625484,0,0,1,0
400976,1,0,0,0
400979,1,0,0,0
271665,0,0,0,1
400985,1,0,0,0
400946,0,1,0,0
401015,0,1,0,0
401005,0,1,0,0
